In [1]:
import requests
import json
import numpy as np

In [2]:
import json
import requests
from collections import defaultdict

In [3]:
from lxml import html
import pandas as pd

In [4]:
df =pd.read_csv("altmetricIDS.csv", header=None)

In [5]:
df.columns = ['Altmetrics_Id']
df.head()

,Altmetrics_Id
0,702709
1,403270
2,2394296
3,632123
4,411425


In [7]:
df["youtube_links"] = ""
df.head()

,Altmetrics_Id,youtube_links
0,702709,
1,403270,
2,2394296,
3,632123,
4,411425,


In [8]:
ids = np.array(df['Altmetrics_Id'])
ids[5255]

1969481

In [9]:
def add_video(alt_id):
    try:
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
        if 'video' in dict(json.loads(response.content)):
            return dict(json.loads(response.content))['video']
    except:
        return ''

In [10]:
param = {"key" : ""}

In [11]:
url_content = "https://www.googleapis.com/youtube/v3/videos?part=snippet&key=&id="

In [12]:
url = "https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics"

In [62]:
from bs4 import BeautifulSoup
for alt_id in ids[5655:5755]:
    mylist = []
    alt_req = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
    alt_rs= json.loads(alt_req.text)
    mylist.append({str(alt_id) : alt_rs})
    y_links = []
    html = requests.get('https://www.altmetric.com/details/' + str(alt_id) + '/video').text
    bs = BeautifulSoup(html)
    possible_links = bs.find_all('a')
    for link in possible_links:
        if link.has_attr('href') and 'youtube' in link.attrs['href']:
            y_links.append(link.attrs['href'])
    youtube_data = []
    for link in y_links:
        param["id"] = link[32:]
        response_statistics = requests.get(url, param)
        print(response_statistics.status_code)
        json_statistics = json.loads(response_statistics.text)
        youtube_data.append(json_statistics)
    mylist.append({'youtube':youtube_data})
    file = open(str(alt_id) + ".txt", "w+")
    json.dump(mylist, file)
    file.close()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [22]:
import os
path = "C:/Users/srika/Youtube_dataSet"
files = os.listdir(path)
len(path)

30

In [33]:
for i in range(len(files)):
    files[i] = os.path.abspath(path + "/" + files[i])

In [37]:
len("C:\\Users\\srika\\Youtube_dataSet\\")

31

In [46]:
df_ = pd.DataFrame()
col = ['cited_by_fbwalls_count', 'cited_by_feeds_count', 'cited_by_gplus_count', 'cited_by_msm_count', 
       'cited_by_posts_count', 'cited_by_rdts_count', 'cited_by_tweeters_count', 'cited_by_videos_count']

In [47]:
stats = ['viewCount', 'likeCount',  'dislikeCount', 'favoriteCount','commentCount']

In [48]:
i=0
for file in files:
    json_file = open(file, "r")
    data = json.load(json_file)
    df_.loc[i, "altmetric_id"] = file[31:-4]
    for c in col:
        try:
            df_.loc[i, c] = data[0][file[31:-4]][c]
        except KeyError:
            df_.loc[i, c] = 0
    i += 1
    json_file.close
    
df_.head() 

,altmetric_id,cited_by_fbwalls_count,cited_by_feeds_count,cited_by_gplus_count,cited_by_msm_count,cited_by_posts_count,cited_by_rdts_count,cited_by_tweeters_count,cited_by_videos_count
0,10025714,14.0,17.0,10.0,108.0,535.0,1.0,304.0,4.0
1,101767,23.0,26.0,6.0,61.0,400.0,1.0,188.0,4.0
2,10282275,26.0,12.0,2.0,47.0,976.0,0.0,834.0,4.0
3,1065504,785.0,5.0,3.0,16.0,1033.0,2.0,146.0,4.0
4,106634,3.0,18.0,2.0,29.0,132.0,0.0,54.0,4.0


In [58]:
df_videos = pd.DataFrame()

In [59]:
i=0
for file in files:
    json_file = open(file, "r")
    data = json.load(json_file)
    for j in range(len(data[1]['youtube'])):
        for stat in stats:
            try:
                df_videos.loc[i, 'altmetric_id'] = file[31:-4]
                df_videos.loc[i, stat] = data[1]['youtube'][j]['items'][0]['statistics'][stat]
            except IndexError:
                df_videos.loc[i, stat] = 0
            except KeyError:
                df_videos.loc[i, stat] = 0
        i += 1
    json_file.close()
df_videos.head()

,altmetric_id,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,10025714,854202,23282,466,0,1108
1,10025714,720,35,1,0,8
2,10025714,2756,279,1,0,44
3,10025714,27,0,0,0,0
4,101767,44688,3287,70,0,195


In [60]:
df_.to_csv("altmetrics_data.csv")

In [61]:
df_videos.to_csv("videos.csv") 

In [ ]:
y_link = url_statistics +"GSqdhz--yR8"
rs_st = requests.get(y_link)
rs_st_d = json.loads(rs_st.text)
rs_st_d

In [ ]:
y_content = url_content + "GSqdhz--yR8"
rs_ct = requests.get(y_link)
rs_ct_d = json.loads(rs_st.text)
rs_ct_d

In [ ]:
social_media_columns = ['cited_by_fbwalls_count', 'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count', 
                        'cited_by_tweeters_count', 'cited_by_videos_count', 'cited_by_wikipedia_count', 
                        'cited_by_accounts_count',]

In [ ]:
cols = ['title', 'doi']

In [ ]:
for i in range(len(ids[:100])):
    rs = requests.get("https://api.altmetric.com/v1/id/" + str(ids[i]))
    if (rs.status_code == 200):
        rs_json = json.loads(rs.text)
        for col in cols:
            try:
                df.loc[i, col] = rs_json[col]
            except KeyError:
                df.loc[i, col] = 0

In [ ]:
df.head()

In [ ]:
df[:100].to_csv(path_or_buf = ".\\sample_dataset.csv", index=False)

In [ ]:
url_comments = "https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=_qUhIZcxNvE&key="

In [ ]:
response = requests.get(url)
print(response.status_code)

In [ ]:
json_data = json.loads(response.text)
json_data.keys()

In [ ]:
json_data["items"][2]['snippet']['topLevelComment']['snippet']

In [ ]:
lc = 0
dc = 0
for j in len(json_data['items']):
    lc + = json_data['items'][j]["likeCount"]
    dc + = json_data['items'][j]["likeCount"]

In [ ]:
# add author information
def add_all_info(alt_id):
    # create the dict
    result = defaultdict(dict)
    try:
        # make the request
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))

        # add the values to the dict
        if 'authors' in dict(json.loads(response.content)):
            result['author_count'] = len(dict(json.loads(response.content))['authors'])
            result['authors'] = ', '.join(dict(json.loads(response.content))['authors'])
        else:
            result['author_count'] = ''
            result['authors'] = ''

         # add the values to the dict
        if 'doi' in dict(json.loads(response.content)):
            result['doi'] = dict(json.loads(response.content))['doi']
        else:
            result['doi'] = ''

        # add the abstract
        if 'abstract' in dict(json.loads(response.content)):
            result['abstract'] = dict(json.loads(response.content))['abstract']
        else:
            result['abstract'] = ''

        # add the pubdate
        if 'published_on' in dict(json.loads(response.content)):
            result['pub_date'] = dict(json.loads(response.content))['published_on']
        else:
            result['pub_date'] = ''

        # add the twitter posts count as of 2018/19
        if 'cited_by_tweeters_count' in dict(json.loads(response.content)):
            result['twitter_count_y18'] = dict(json.loads(response.content))['cited_by_tweeters_count']
        else:
            result['twitter_count_y18'] = ''

        # add the title of the article
        if 'title' in dict(json.loads(response.content)):
            result['title'] =  dict(json.loads(response.content))['title']
        else:
            result['title'] = ''

        # return the result
        return result
    except:
        return result

In [ ]:
r = add_all_info(702709 )

In [ ]:
r.keys()

In [ ]:
rs_ = requests.get("https://api.altmetric.com/v1/id/" + str(2394296))

In [ ]:
rs_js = json.loads(rs_.text)

In [ ]:
rs_js

In [ ]:
fl = open("temp.txt", 'w+')

In [ ]:
json.dump(rs_js, fl)

In [ ]:
rs_js['cited_by_accounts_count']

In [ ]:
rs_js['cited_by_videos_count']

In [ ]:
response_statistics = requests.get(url_statistics)
print(response.status_code)

In [ ]:
json_statistics = json.loads(response_statistics.text)
json_statistics.keys()

In [ ]:
json_statistics['items']